In [2]:
!pip install dotenv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [67 lines of output]
  C:\Users\prove\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
    warnings.warn(
    error: subprocess-exited-with-error
  
    python setup.py egg_info did not run successfully.
    exit code: 1
  
    [16 lines of output]
    Traceback (most recent call last):
      File "<string>", line 2, in <module>
      File "<pip-setuptools-caller>", line 14, in <module>
      File "C:\Users\prove\AppData\Local\Temp\pip-wheel-mcvgpbti\distribute_053daaf7e241455f8d8e89e8a066de3c\setuptools\__init__.py", line 2, in <module>
        from setuptools.extension import Extension, Library
      File "C:\Users\prove\AppData\Local\Temp\pip-wheel-mcvgpbti\distribute_053daaf7e241455f8d8e89e8a066de3c\setuptools\ex

In [1]:
import os
import json
import requests
import pandas as pd

import pytz
import time
from datetime import datetime

from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
load_dotenv()
TOKEN = os.getenv('JUSBRASIL_TOKEN')

In [18]:
def get_datetime_now(timezone="America/Sao_Paulo"):
    return datetime.now(tz=pytz.timezone("America/Sao_Paulo"))

In [19]:
def background_check(cpf, pagination=None, route_path='background_check/search', token=TOKEN):
    headers = {'Content-Type': 'application/json'}
    
    url = f'https://op.digesto.com.br/api/{route_path}'
    
    payload = {'document': f'{cpf.zfill(11)}'}
    if pagination and 'size' in pagination.keys() and 'cursor' in pagination.keys():
        payload.update(pagination)

    response = requests.request('POST', url, headers=headers, json=payload, auth=(f'{token}', ''))
    
    parsed = None
    
    if response.status_code == 200:
        parsed = json.loads(response.text)
    
    return parsed

In [20]:
def get_execution_time(parsed):
    execution_time, extensions_execution_time = -1, -1
    
    if parsed and parsed.get('extensions'):
        execution_time, extensions_execution_time = parsed.get('extensions').get('executionTime', -1), parsed.get('extensions').get('extensionsExecutionTime', -1)
        
    return execution_time, extensions_execution_time

In [21]:
def get_cursor(parsed):
    return parsed.get('data').get('root').get('crmLawsuitsByPersonId').get('pageInfo').get('endCursor')

In [22]:
def get_has_next_page(parsed):
    return parsed.get('data').get('root').get('crmLawsuitsByPersonId').get('pageInfo').get('hasNextPage')

In [23]:
def get_next_page_info(parsed, n_results_per_page = 10):
    assert(n_results_per_page <= 500)
    dict_pagination = None
    
    if parsed:
        cursor = get_cursor(parsed)

        if get_has_next_page(parsed):
            dict_pagination = {'pagination': {'size': n_results_per_page, 'cursor': cursor}}
        
    return dict_pagination

In [24]:
def normalize_driver_data(parsed):
    df = None
    if parsed:
        df = pd.json_normalize(parsed.get('data').get('root').get('crmLawsuitsByPersonId').get('edges'), sep='_', max_level=2)
    return df

In [25]:
def pipeline_search_driver_lawsuits(cpf, sleep_time_among_requests = 0.5):
    total_execution_time = 0
    total_extensions_execution_time = 0
    
    has_next_page = True
    last_cursor = None
    cursor = None
    dict_pagination = None
    
    df_all_data_driver = pd.DataFrame()

    page = 1
    while has_next_page:
        parsed = background_check(f'{cpf.zfill(11)}', pagination=dict_pagination)
        
        execution_time, extensions_execution_time = get_execution_time(parsed)
        total_execution_time += execution_time
        extensions_execution_time += extensions_execution_time
        
        df_page_data_driver = normalize_driver_data(parsed)
        
        df_page_data_driver['dt_search_for_lawsuits'] = pd.to_datetime(get_datetime_now().replace(tzinfo=None))
        df_page_data_driver['page'] = page
        df_page_data_driver['execution_time'] = execution_time
        df_page_data_driver['extensions_execution_time'] = extensions_execution_time
        
        df_all_data_driver = pd.concat([df_all_data_driver, df_page_data_driver], axis=0)

        dict_pagination = get_next_page_info(parsed, n_results_per_page=10)

        last_cursor = cursor
        cursor = get_cursor(parsed)
        has_next_page = get_has_next_page(parsed)

        if last_cursor == cursor:
            break
        
        page += 1
        time.sleep(sleep_time_among_requests)

    df_all_data_driver['total_execution_time'] = total_execution_time
    df_all_data_driver['total_extensions_execution_time'] = extensions_execution_time
    
    return df_all_data_driver.drop_duplicates()

In [26]:
def get_lawsuit_details(cnj, tipo_numero=5, route_path='tribproc/', token=TOKEN):
    headers = {'Content-Type': 'application/json'}
    
    url = f'https://op.digesto.com.br/api/{route_path}/{cnj}?tipo_numero={tipo_numero}'

    response = requests.request('GET', url, headers=headers, auth=(f'{token}', ''))
    
    parsed = None
    
    if response.status_code == 200:
        parsed = json.loads(response.text)
    
    return parsed

In [27]:
def normalize_lawsuit_data(parsed):
    df = None
    if parsed:
        df = pd.json_normalize(parsed, sep='_')
    return df

In [28]:
def pipeline_search_lawsuit_details(cnj):
    df_lawsuit_details = pd.DataFrame()
    
    lawsuit_details = get_lawsuit_details(f'{cnj.strip()}')
    if lawsuit_details:
        df_lawsuit_details = normalize_lawsuit_data(lawsuit_details)
        df_lawsuit_details['dt_search_lawsuit_details'] = pd.to_datetime(get_datetime_now().replace(tzinfo=None))
    
    return df_lawsuit_details

In [29]:
def pipeline_search_lawsuits_details(list_cnjs):
    df_lawsuits_details = pd.DataFrame()
    
    for cnj in list_cnjs:
        df_lawsuit_details = pipeline_search_lawsuit_details(cnj)
        df_lawsuits_details = pd.concat([df_lawsuits_details, df_lawsuit_details], axis=0)
    
    return df_lawsuits_details

In [30]:
cpfs = ['11291122680',
 '1301501182',
 '1454470909',
 '15030504915',
 '17112010144',
 '21482348896',
 '2280810921',
 '2387430123',
 '2428273906',
 '26200575827',
 '27977980827',
 '31467636053',
 '3207819176',
 '3215460246',
 '33003625802',
 '33989923900',
 '34199155864',
 '34339434000',
 '36452356801',
 '3749404950',
 '4063153100',
 '4294169974',
 '47522828104',
 '4981109903',
 '4996344113',
 '50375750100',
 '51119765072',
 '51460858115',
 '53392841072',
 '57156417134',
 '61967521972',
 '62515624200',
 '632083182',
 '63982790182',
 '65533151915',
 '7097699469',
 '7414637921',
 '7633237805',
 '77939271172',
 '81164700197',
 '82753946868',
 '87037459100',
 '902957066',
 '90693353104',
 '91225124115',
 '91998344053',
 '93740670053',
 '95800930830',
 '96913592',
 '98037838153']

cpfs_not_found = []
cpfs_not_found_details = []
df_all_drivers = pd.DataFrame()

start_time = time.time()
for cpf in cpfs:    
    df_driver_lawsuits = pipeline_search_driver_lawsuits(f'{cpf}')
    
    if not df_driver_lawsuits.empty:
        list_cnjs = df_driver_lawsuits['node_cnjNumber_number'].tolist()
        df_lawsuits_details = pipeline_search_lawsuits_details(list_cnjs)
        if not df_lawsuits_details.empty and 'numero' in df_lawsuits_details.columns:
            final_df = df_driver_lawsuits.merge(df_lawsuits_details, left_on='node_cnjNumber_number', right_on='numero')
            df_all_drivers = pd.concat([df_all_drivers, final_df], axis=0)
        else:
            cpfs_not_found_details.append(cpf)
    else:
        cpfs_not_found.append(cpf)

end_time = (time.time() - start_time)
print("--- %s seconds ---" % end_time)

df_all_drivers['total_exp_time'] = end_time
df_all_drivers.to_excel('drivers_lawsuits_details.xlsx', index=False)

df_all_drivers.head()

C:\Users\prove\AppData\Local\Temp\ipykernel_12800\2382115063.py:65: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_all_drivers = pd.concat([df_all_drivers, final_df], axis=0)
C:\Users\prove\AppData\Local\Temp\ipykernel_12800\2382115063.py:65: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_all_drivers = pd.concat([df_all_drivers, final_df], axis=0)
C:\Users\prove\AppData\Local\Temp\ipykernel_12800\2382115063.py:65: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_all_drivers = pd.concat([df_all_drivers, final_df], axis=0)
C:\Users\prove\AppData\Local\Temp\ipykernel_12800\2382115063.py:65: FutureWarning:

--- 107.5083520412445 seconds ---


,matchRelatedPeople_name,matchRelatedPeople_personId,matchRelatedPeople_role_normalized,matchRelatedPeople_role_rawValue,node_cnjNumber_number,node_court_rawValue,node_distributionDate,node_distributionType,node_district,node_forum,...,segredo_justica,arquivado,classes,acessos,uf,criadoEm,partes,dt_search_lawsuit_details,status_op,total_exp_time
0,Pedro Henrique Felipe de Faria,4994653802748477657:28,Réu,PASSIVO,1005536-37.2021.8.11.0040,TJMT,1648695600000,None,CUIABÁ CRIMINAL,None,...,False,False,"[Receptação, Furto]",[2021-08-02 17:16:52],MT,2021-08-02T17:15:36,"[[681294290, 63071372, DINAMILTON BARROS DA SI...",2022-10-05 00:52:26.088431,NaN,107.508352
1,Pedro Henrique Felipe de Faria,4994653802748477657:28,Réu,RÉU,0168937-32.2021.8.13.0702,TJMG,1629774000000,Sorteio,UBERLÂNDIA/MG,None,...,False,False,[],[2021-08-26 06:56:39],MG,2021-08-26T06:56:39,"[[690072348, 54929050, PEDRO HENRIQUE FELIPE D...",2022-10-05 00:52:27.473927,NaN,107.508352
2,Pedro Henrique Felipe de Faria,4994653802748477657:28,None,AD,1013493-15.2021.8.11.0000,TJMT,1627354800000,Sorteio,None,None,...,False,False,"[Prisão Preventiva, Furto Qualificado, Recepta...","[2022-07-22 15:10:11, 2021-08-02 22:00:21]",MT,2021-08-02T22:00:21,"[[681415741, 13629174, JUÍZO DA 2ª VARA CRIMIN...",2022-10-05 00:52:28.385458,NaN,107.508352
0,Sabino Xavier dos Santos Junior,1464604427797647554:29,Réu,Polo Passivo,0005656-90.2022.8.16.0083,TJPR,1662692400000,None,Francisco Beltrão,Vara Criminal,...,False,False,"[Citação, Apropriação indébita]",[2022-09-09 21:24:28],PR,2022-09-09T21:24:28,"[[888809049, 19094, Ministério Público Estadua...",2022-10-05 00:52:32.107840,NaN,107.508352
1,Sabino Xavier dos Santos Junior,1464604427797647554:29,Réu,Polo Passivo,0005656-90.2022.8.16.0083,TJPR,1662692400000,None,Francisco Beltrão,Vara Criminal,...,False,False,"[Citação, Apropriação indébita]",[2022-09-09 21:24:28],PR,2022-09-09T21:24:28,"[[888809049, 19094, Ministério Público Estadua...",2022-10-05 00:52:40.305004,NaN,107.508352


In [46]:
df_all_drivers.drop_duplicates(subset=['matchRelatedPeople_name']).columns

Index(['matchRelatedPeople_name', 'matchRelatedPeople_personId',
       'matchRelatedPeople_role_normalized',
       'matchRelatedPeople_role_rawValue', 'node_cnjNumber_number',
       'node_court_rawValue', 'node_distributionDate', 'node_distributionType',
       'node_district', 'node_forum', 'node_id', 'node_justiceSecret',
       'node_normalizedNature', 'node_value', 'dt_search_for_lawsuits', 'page',
       'execution_time', 'extensions_execution_time', 'total_execution_time',
       'total_extensions_execution_time', 'regional_cnj', 'numeroAlternativo',
       'vara', 'classeNatureza', 'movs', 'distribuicaoTipo', 'situacao',
       'juiz', 'area', 'assuntoExtra', 'liminar', 'audiencias',
       'vara_original', 'extinto', 'gratuita', 'valor', 'alteradoEm',
       'fonte_sistema', 'foro', 'situacao_situacaoID', 'sentencaData',
       'instancia', 'processosRelacionados', 'numero', 'customs', 'flag',
       'comarca_cnj', 'comarca', 'tribunalID', 'classeNatureza_dg', 'foro_cnj',
  

In [47]:
pd.set_option('display.max_columns', None)
df_all_drivers.drop_duplicates(subset=['matchRelatedPeople_name'])#.shape

,matchRelatedPeople_name,matchRelatedPeople_personId,matchRelatedPeople_role_normalized,matchRelatedPeople_role_rawValue,node_cnjNumber_number,node_court_rawValue,node_distributionDate,node_distributionType,node_district,node_forum,node_id,node_justiceSecret,node_normalizedNature,node_value,dt_search_for_lawsuits,page,execution_time,extensions_execution_time,total_execution_time,total_extensions_execution_time,regional_cnj,numeroAlternativo,vara,classeNatureza,movs,distribuicaoTipo,situacao,juiz,area,assuntoExtra,liminar,audiencias,vara_original,extinto,gratuita,valor,alteradoEm,fonte_sistema,foro,situacao_situacaoID,sentencaData,instancia,processosRelacionados,numero,customs,flag,comarca_cnj,comarca,tribunalID,classeNatureza_dg,foro_cnj,tribunal,processoID,distribuicaoData,segredo_justica,arquivado,classes,acessos,uf,criadoEm,partes,dt_search_lawsuit_details,status_op,total_exp_time
0,Pedro Henrique Felipe de Faria,4994653802748477657:28,Réu,PASSIVO,1005536-37.2021.8.11.0040,TJMT,1648695600000,None,CUIABÁ CRIMINAL,None,Q1JNTGF3c3VpdDp7Imxhd3N1aXRJZCI6IjFiYmFkY2NjLW...,False,PETIÇÃO CRIMINAL,0.0,2022-10-05 00:52:24.715894,1,139,0,139,0,False,PETIÇÃO CRIMINAL 1005536-37.2021.8.11.0040,2,PETICAO CRIMINAL,"[[2021-07-31, Andamento - Juntada, JUNTADA - J...",SORTEIO,None,None,Cível,"Receptação,Furto",None,"[[2021-07-26 17:10:00, 2ª VARA CRIMINAL DE SOR...",2,0.0,None,NaN,2021-08-02T17:15:36,TJMT PJEMNI,VARA CRIMINAL,None,None,1.0,[],1005536-37.2021.8.11.0040,[],1.0,Sorriso,SORRISO,47.0,1.0,Sorriso,TJMT,455850892.0,2021-06-16,False,False,"[Receptação, Furto]",[2021-08-02 17:16:52],MT,2021-08-02T17:15:36,"[[681294290, 63071372, DINAMILTON BARROS DA SI...",2022-10-05 00:52:26.088431,NaN,107.508352
0,Sabino Xavier dos Santos Junior,1464604427797647554:29,Réu,Polo Passivo,0005656-90.2022.8.16.0083,TJPR,1662692400000,None,Francisco Beltrão,Vara Criminal,Q1JNTGF3c3VpdDp7Imxhd3N1aXRJZCI6IjdhYTFmZjBiLW...,False,OUTROS,NaN,2022-10-05 00:52:29.873541,1,140,0,252,0,True,0005656-90.2022.8.16.0083 Carta Precatória Cri...,CRIMINAL,CARTA PRECATORIA CRIMINAL,"[[2022-09-09, Distribuicao - Distribuido, DIST...",None,None,JANAINA MONIQUE ZANELLATO ALBINO,Cível,"Citação,Apropriação indébita",None,[],CRIMINAL,0.0,None,NaN,2022-09-09T21:24:28,TJPR PROJUDI,VARA CRIMINAL,None,None,1.0,[],0005656-90.2022.8.16.0083,[],1.0,Francisco Beltrão,FRANCISCO BELTRAO,52.0,1.0,Fórum de Francisco Beltrão,TJPR,502416205.0,2022-09-09,False,False,"[Citação, Apropriação indébita]",[2022-09-09 21:24:28],PR,2022-09-09T21:24:28,"[[888809049, 19094, Ministério Público Estadua...",2022-10-05 00:52:32.107840,NaN,107.508352
0,Diari Andre Marcante,x7211522248304054776:20,Réu,Polo Passivo,1002881-31.2020.4.01.3600,TRF01,1582772400000,None,SJMT,None,Q1JNTGF3c3VpdDp7Imxhd3N1aXRJZCI6IjIxOGVmNWRkLW...,False,AÇÃO PENAL - PROCEDIMENTO ORDINÁRIO,NaN,2022-10-05 00:52:50.795083,1,108,0,108,0,None,AÇÃO PENAL - PROCEDIMENTO ORDINÁRIO 1002881-31...,7,ACAO PENAL - PROCEDIMENTO ORDINARIO,"[[2022-03-10, Andamento - Recebido, RECEBIDO -...",DEPENDENCIA,None,None,Cível,DIREITO PENAL (287) - Crimes contra a Fé Púb...,None,[],7,0.0,None,NaN,2022-03-19T20:39:49,TRF1 PJE,VARA FEDERAL CRIMINAL,None,None,1.0,"[{'tribunal': 'TRF1', 'natureza': 'AUTO DE PRI...",1002881-31.2020.4.01.3600,[],1.0,Mato Grosso,SJMT,8.0,1.0,Cuiabá,TRF1,478121452.0,2020-02-27,False,False,[DIREITO PENAL (287) - Crimes contra a Fé Pú...,[2022-03-19 20:39:50],None,2022-03-19T20:39:49,"[[777220083, 30343708, DIARI ANDRE MARCANTE, D...",2022-10-05 00:52:52.814366,NaN,107.508352
0,Gelcino Martins dos Santos,8356481368706895169:25,Indiciado,INDICIADO,0019003-45.2000.8.26.0032,TJSP,957754800000,Livre,Foro de Araçatuba,Foro de Araçatuba,Q1JNTGF3c3VpdDp7Imxhd3N1aXRJZCI6ImJiMjgwYWZjLW...,False,OUTROS,NaN,2022-10-05 00:52:59.563420,1,75,0,75,0,True,Carta Precatória Criminal 0019003-45.2000.8.26...,3,CARTA PRECATORIA CRIMINAL,"[[2021-10-06, Andamento - Saneamento da base d...",LIVRE,ARQUIVADO,DANILO BRAIT,Cível,,None,[],3,0.0,None,NaN,2022-07-

In [31]:
cpfs_not_found, cpfs_not_found_details

(['1301501182',
  '15030504915',
  '17112010144',
  '21482348896',
  '2387430123',
  '2428273906',
  '26200575827',
  '31467636053',
  '3215460246',
  '33003625802',
  '33989923900',
  '34199155864',
  '36452356801',
  '3749404950',
  '4063153100',
  '4294169974',
  '47522828104',
  '50375750100',
  '51119765072',
  '51460858115',
  '632083182',
  '65533151915',
  '7097699469',
  '7414637921',
  '81164700197',
  '82753946868',
  '87037459100',
  '902957066',
  '90693353104',
  '91998344053',
  '95800930830',
  '96913592',
  '98037838153'],
 ['57156417134'])

In [37]:
len(cpfs_not_found)

33

In [32]:
import pandas as pd

In [33]:
df_escavador = pd.read_csv('data/results_escavador.csv')

In [34]:
df_escavador = df_escavador[['cod_cpf', 'result_escavador_manual']]

In [36]:
df_escavador['cod_cpf'] = df_escavador['cod_cpf'].astype(str).str.zfill(11)
df_escavador['result'] = df_escavador['result_escavador_manual'].apply(lambda x: 1 if x == 'CRIMINAL' else 0)

In [9]:
df_escavador.groupby('result').count()

,cod_cpf,result_escavador_manual
result,,
0,25,25
1,25,25


In [ ]:
cpfs_encontrados_jusbrasil = set(cpfs) - set(cpfs_not_found + cpfs_not_found_details)

In [ ]:
## Casos encontrados pelo escavador, mas não encontrados pela Jusbrasil

In [70]:
df_escavador[(df_escavador['result']==1)].shape[0], len(cpfs_encontrados_jusbrasil)

(25, 16)

In [ ]:
df_escavador[(df_escavador['result']==1) & (df_escavador['cod_cpf'].isin(cpfs_encontrados_jusbrasil))]['cod_cpf']

In [ ]:
df_escavador[(df_escavador['result']==0) & (df_escavador['cod_cpf'].isin(cpfs_encontrados_jusbrasil))]['cod_cpf']

In [68]:
df_escavador[(df_escavador['result']==1) & (~df_escavador['cod_cpf'].isin(cpfs_encontrados_jusbrasil))]['cod_cpf']

18    31467636053
19    00096913592
21    98037838153
22    04294169974
24    02280810921
25    51119765072
27    02428273906
28    00632083182
37    57156417134
38    17112010144
39    65533151915
40    02387430123
44    04063153100
45    07097699469
47    33003625802
48    36452356801
49    21482348896
Name: cod_cpf, dtype: object

In [79]:
cpfs_nao_encontrados_jusbrasil = df_escavador[(df_escavador['result']==1) & (~df_escavador['cod_cpf'].isin(cpfs_encontrados_jusbrasil))]['cod_cpf'].tolist()

In [80]:
for cpf in cpfs_nao_encontrados_jusbrasil:
    parsed = background_check(cpf=cpf)
    if not parsed.get('data').get('root').get('crmLawsuitsByPersonId').get('edges'):
        print(f'{cpf} - nada encontrado.')

31467636053 - nada encontrado.
00096913592 - nada encontrado.
98037838153 - nada encontrado.
04294169974 - nada encontrado.
51119765072 - nada encontrado.
02428273906 - nada encontrado.
00632083182 - nada encontrado.
17112010144 - nada encontrado.
65533151915 - nada encontrado.
02387430123 - nada encontrado.
04063153100 - nada encontrado.
07097699469 - nada encontrado.
33003625802 - nada encontrado.
36452356801 - nada encontrado.
21482348896 - nada encontrado.


In [81]:
background_check(cpf='57156417134')

{'data': {'root': {'crmLawsuitsByPersonId': {'edges': [{'matchRelatedPeople': {'name': 'Wilmar Vidal dos Santos',
       'personId': '988063910266010823:22',
       'role': {'normalized': None, 'rawValue': 'Réu(s):'}},
      'node': {'cnjNumber': {'number': '0004813-41.2013.8.11.0064'},
       'court': {'rawValue': 'TJMT'},
       'distributionDate': 1553752800000,
       'distributionType': None,
       'district': None,
       'forum': None,
       'id': 'Q1JNTGF3c3VpdDp7Imxhd3N1aXRJZCI6ImQ5YmJiMThhLWF1dG9tYXRpYzowLTAwMDQ4MTM0MTIwMTM4MTEwMDY0LWxhd3N1aXQifQ==',
       'justiceSecret': False,
       'normalizedNature': 'AÇÃO PENAL - PROCEDIMENTO ORDINÁRIO',
       'value': None}},
     {'matchRelatedPeople': {'name': 'Wilmar Vidal dos Santos',
       'personId': '988063910266010823:22',
       'role': {'normalized': 'Réu', 'rawValue': 'Réu'}},
      'node': {'cnjNumber': {'number': '0001880-74.2016.8.11.0037'},
       'court': {'rawValue': 'TJMT'},
       'distributionDate': 1459393200

In [82]:
get_lawsuit_details(cnj='0004813-41.2013.8.11.0064')

{'status_op': 'Processo não encontrado'}

In [83]:
get_lawsuit_details(cnj='0001880-74.2016.8.11.0037')

{'status_op': 'Processo não encontrado'}